# Artifical Neural Networks & Deep Learning
# Homework 1 - Image Classification

**Developement Team:**
- Acquati Marco - 10583134 
- Brugali Giorgio - 10794550
- Puoti Francesco - 10595640 


# *1. Data acquisition and augmentation*

> This topic has been issued with annotations as the code flows down, to better clarify the correspondence between the explanations and the code snippets 


# *2. Model overview*


> ***2.1. Features' Extraction***

>> The feature extractor is composed by *5 blocks*, each composed by:
  - one convolutional part
  - one  MaxPooling layer.
  
>>The convolutional part has a variable number of convolutional layers: 2 Conv2D layers for the first two blocks and 3 Conv2D layers for the remaining ones.
This choice was dictated with the intention of reducing the parameters' number in the network.Our idea was to start with small filters 3x3 because of the need to detail the features' extraction at the beginning. Afterwards, instead of increasing the filters' size, to avoid having possible distorted features that could result too effective in the learning process, we decided to augment the number of convolutional layers.
Regarding the activation function, we chose the ReLU activation function as it is more suitable for the classification problem.

>>BatchNormalization has been involved in the features' extraction part in order to both improve the stability of our network and to reduce covariance shift, the latter resulting in improving the training velocity as well. 

>>The pool size has been set to 3x3 with stride 2x2 to favor the overlapping: it has been demonstrated that the overlapping pooling areas reduce the likelihood of the network to overfit. 

> ***2.2. BottleNeck Layer***
>> In order to reduce the computational load of the network, 
the number of the features extractor’s output channels is reduced by adding a 1x1 convolutional layer before feeding the output to the classifier.

> ***2.3. Classifier***
>> - one flatten layer
>> - two dense hidden layers with 2048 neurons each
>> - the output layer with the SOFTMAX activation function and three classes
>> It's worth highlighting the use of weight initialization (HeNormal distribution), which aims at improving the network speed, avoiding too many zeroes in the kernels at the beginning of the learning process.
>> Moreover, weight decay has been implemented to reduce overfitting in the Dense layers.
>> Both BatchNormalization and ReLU have been used for the same purpose as in the features' extraction part.

> ***2.4. Optimizer & LossFunction***
>> - Adam, with a starting learning rate of 1e-3 and amsgrad = True to have an adaptive learning rate, so as to prevent the network from being stuck on a suboptimal solution.
>> - Loss function : Categorical Crossentropy.

> ***2.5. Further information about the implemention process***
>> No EarlyStopping has been used in the final model as, after some trials, such model got stopped even though the learning process would have subsequently led to noteworthy improvements.
>> Model checkpoints could not be implemented due to memory shortage.


In [1]:
import numpy as np 
import pandas as pd 
import tensorflow as tf

SEED = 1234
tf.random.set_seed(SEED)

In [2]:
import os
import json
import operator

# Since the training images were not divided in subfolders, 
#     we had to manage the data acquisition by means of data frame. 
# First, we acquired the images' paths an we sorted them in order to create 
#     a correspondence between each image and its label stored in the json file.
#-------------------------------------------------------------------------------

X = [] #list of images' paths
for dirname, _, filenames in os.walk('../input/artificial-neural-networks-and-deep-learning-2020/MaskDataset/training'):
    filenames.sort()
    for filename in filenames:
           X.append(os.path.join(dirname, filename))

with open('/kaggle/input/artificial-neural-networks-and-deep-learning-2020/MaskDataset/train_gt.json') as f:

 data = json.load(f)

data = sorted(data.items(), key=operator.itemgetter(0))

y = [] #list of target labels
for i in range(len(data)):
    y.append(str(data[i][1])) #il dataframe vuole delle string

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# We decided to apply data augmentation on the training set not because of the data shortage
# but in order to make our model more flexibile on recognizing objects in different positions and dimensions.
#------------------------------------------------------------------------------------------------------------

train_data_gen = ImageDataGenerator(rotation_range=10,
                                    width_shift_range=10,
                                    height_shift_range=10,
                                    zoom_range=0.3,
                                    horizontal_flip=True,
                                    vertical_flip=True,
                                    fill_mode='nearest',
                                    rescale=1./255)


# No data aumentation has been applied on validation set, since we want to have the images meant for validation 
# similar to the test images to find out the features of our model
#--------------------------------------------------------------------------------------------------------------
valid_data_gen = ImageDataGenerator(rescale=1./255)

In [4]:
from sklearn.model_selection import train_test_split

# Data set split in training and validation sets 
# with the latter having a size equal to the 20% of the entire data set.
#-----------------------------------------------------------------------
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25)


# Generation of training and validation dataframes using pandas' library
#------------------------------------------------------------------------
dataframe_train = pd.DataFrame({"input": X_train, "target": y_train})
dataframe_valid = pd.DataFrame({"input": X_valid, "target": y_valid})


In [5]:
# Batch size
bs = 32

# img shape
img_h = 256
img_w = 256

num_classes = 3

classes = ['0', '1', '2']
clmode = "rgb"

# Creation of the DataFrameIterators yielding tuples of (x, y) 
# where x is a numpy array containing a batch of images with shape (batch_size, *target_size, channels) 
# and y is a numpy array of corresponding labels.
#------------------------------------------------------------------------------------------------------

train_datagen = train_data_gen.flow_from_dataframe(
      dataframe = dataframe_train,
      directory = './',
      x_col = "input",
      y_col = "target",
      target_size = (img_h, img_w),
      color_mode = clmode,
      classes = classes,
      class_mode = "categorical",
      batch_size = bs,
      shuffle = True,
      seed = SEED
)

valid_datagen = valid_data_gen.flow_from_dataframe(
      dataframe = dataframe_valid,
      directory = './',
      x_col = "input",
      y_col = "target",
      target_size = (img_h, img_w),
      color_mode = clmode,
      classes = classes,
      class_mode = "categorical",
      batch_size = bs,
      shuffle = True,
      seed = SEED
)

Found 4210 validated image filenames belonging to 3 classes.
Found 1404 validated image filenames belonging to 3 classes.


In [8]:
# Create Dataset objects
# ----------------------

# Training
train_dataset = tf.data.Dataset.from_generator(lambda: train_datagen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes])) #None for dynamic bs, 
train_dataset = train_dataset.repeat()

# Validation
# ----------
valid_dataset = tf.data.Dataset.from_generator(lambda: valid_datagen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))
valid_dataset = valid_dataset.repeat()

In [18]:
#----------CALLBACKS' SELECTION--------
#--------------------------------------

from datetime import datetime
callbacks = []
now = datetime.now().strftime('%b%d_%H-%M-%S')

exp_dir = './logs'
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)

exp_dir = os.path.join(exp_dir, '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)

# Model checkpoint
# ----------------
ckpt_dir = os.path.join(exp_dir, 'checkpoint')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)    
ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=(ckpt_dir + '/model_' + 'cp_{epoch:02d}.h5'), 
                                            monitor='val_accuracy', verbose=1,
                                            save_best_only=True, mode='max') 
callbacks.append(ckpt_callback)


# Early Stopping
# --------------
es = False
if es:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    callbacks.append(es_callback)

In [19]:
import operator
    
model = tf.keras.Sequential()

depth = 5
start_f = 64
ker_sz = (3,3)
ker_str = (1,1)
pool_sz = (3,3)

initializer = tf.keras.initializers.he_normal(seed=SEED)
regularizer = tf.keras.regularizers.l2(0.001) #weight decay
dpout = 0.5

convNumb = 2

# Features extraction
# -------------------
for i in range(depth):

    if i == 0:
        input_shape = [img_h, img_w, 3]
    else:
        input_shape=[None]
        
    if i == 2:
        convNumb += 1 #increasing number of Conv2D layer in a block
        
    for j in range(convNumb):
        model.add(tf.keras.layers.Conv2D(filters=start_f, 
                                 kernel_size=ker_sz,
                                 strides=ker_str,
                                 padding='same',
                                 input_shape=input_shape))        
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.MaxPool2D(pool_sz, strides = 2))
    if start_f < 512:
        start_f *= 2

#Bottle neck layer
#-----------------
model.add(tf.keras.layers.Conv2D(filters=start_f/2, 
                              kernel_size=(1,1),
                              strides=ker_str,
                              padding='same',
                              input_shape=input_shape))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.ReLU())     

#Classifier
#----------
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=2048, 
                                kernel_regularizer = regularizer,
                                kernel_initializer = initializer))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.ReLU()) 
model.add(tf.keras.layers.Dropout(dpout))
model.add(tf.keras.layers.Dense(units=2048,
                                kernel_regularizer=regularizer, 
                                kernel_initializer = initializer))
model.add(tf.keras.layers.ReLU())
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

In [20]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3, amsgrad=True), 
              loss=tf.keras.losses.CategoricalCrossentropy(), 
              metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 256, 256, 64)      1792      
_________________________________________________________________
batch_normalization_15 (Batc (None, 256, 256, 64)      256       
_________________________________________________________________
re_lu_16 (ReLU)              (None, 256, 256, 64)      0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 256, 256, 64)      36928     
_________________________________________________________________
batch_normalization_16 (Batc (None, 256, 256, 64)      256       
_________________________________________________________________
re_lu_17 (ReLU)              (None, 256, 256, 64)      0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 127, 127, 64)     

In [ ]:
model.fit(x=train_dataset,
          epochs=200,
          steps_per_epoch=len(train_datagen),
          validation_data=valid_dataset,
          validation_steps=len(valid_datagen),
          callbacks=callbacks
         )

Epoch 1/200
132/132 [==============================] - ETA: 0s - loss: 8.9970 - accuracy: 0.3444
Epoch 00001: val_accuracy improved from -inf to 0.33120, saving model to ./logs/_Nov22_10-44-52/checkpoint/model_cp_01.h5
132/132 [==============================] - 144s 1s/step - loss: 8.9970 - accuracy: 0.3444 - val_loss: 6.6715 - val_accuracy: 0.3312
Epoch 2/200
132/132 [==============================] - ETA: 0s - loss: 5.2488 - accuracy: 0.3366
Epoch 00002: val_accuracy improved from 0.33120 to 0.34544, saving model to ./logs/_Nov22_10-44-52/checkpoint/model_cp_02.h5
132/132 [==============================] - 143s 1s/step - loss: 5.2488 - accuracy: 0.3366 - val_loss: 4.0577 - val_accuracy: 0.3454
Epoch 3/200
132/132 [==============================] - ETA: 0s - loss: 3.3922 - accuracy: 0.3347
Epoch 00003: val_accuracy did not improve from 0.34544
132/132 [==============================] - 142s 1s/step - loss: 3.3922 - accuracy: 0.3347 - val_loss: 2.8104 - val_accuracy: 0.3340
Epoch 4/200

132/132 [==============================] - ETA: 0s - loss: 0.4974 - accuracy: 0.8333
Epoch 00027: val_accuracy did not improve from 0.81695
132/132 [==============================] - 150s 1s/step - loss: 0.4974 - accuracy: 0.8333 - val_loss: 0.8698 - val_accuracy: 0.6688
Epoch 28/200
132/132 [==============================] - ETA: 0s - loss: 0.4928 - accuracy: 0.8302
Epoch 00028: val_accuracy did not improve from 0.81695
132/132 [==============================] - 147s 1s/step - loss: 0.4928 - accuracy: 0.8302 - val_loss: 0.6392 - val_accuracy: 0.7671
Epoch 29/200
132/132 [==============================] - ETA: 0s - loss: 0.5191 - accuracy: 0.8273
Epoch 00029: val_accuracy improved from 0.81695 to 0.82906, saving model to ./logs/_Nov22_10-44-52/checkpoint/model_cp_29.h5
132/132 [==============================] - 151s 1s/step - loss: 0.5191 - accuracy: 0.8273 - val_loss: 0.5069 - val_accuracy: 0.8291
Epoch 30/200
132/132 [==============================] - ETA: 0s - loss: 0.4638 - accurac

Epoch 55/200
132/132 [==============================] - ETA: 0s - loss: 0.3968 - accuracy: 0.8765
Epoch 00055: val_accuracy did not improve from 0.87749
132/132 [==============================] - 137s 1s/step - loss: 0.3968 - accuracy: 0.8765 - val_loss: 0.4589 - val_accuracy: 0.8340
Epoch 56/200
132/132 [==============================] - ETA: 0s - loss: 0.3466 - accuracy: 0.8914
Epoch 00056: val_accuracy did not improve from 0.87749
132/132 [==============================] - 139s 1s/step - loss: 0.3466 - accuracy: 0.8914 - val_loss: 0.5370 - val_accuracy: 0.8170
Epoch 57/200
132/132 [==============================] - ETA: 0s - loss: 0.3852 - accuracy: 0.8793
Epoch 00057: val_accuracy did not improve from 0.87749
132/132 [==============================] - 140s 1s/step - loss: 0.3852 - accuracy: 0.8793 - val_loss: 0.5057 - val_accuracy: 0.8212
Epoch 58/200
132/132 [==============================] - ETA: 0s - loss: 0.3761 - accuracy: 0.8874
Epoch 00058: val_accuracy did not improve from 

132/132 [==============================] - ETA: 0s - loss: 0.3179 - accuracy: 0.9107
Epoch 00083: val_accuracy did not improve from 0.88818
132/132 [==============================] - 132s 999ms/step - loss: 0.3179 - accuracy: 0.9107 - val_loss: 0.4551 - val_accuracy: 0.8447
Epoch 84/200
132/132 [==============================] - ETA: 0s - loss: 0.2942 - accuracy: 0.9088
Epoch 00084: val_accuracy did not improve from 0.88818
132/132 [==============================] - 132s 999ms/step - loss: 0.2942 - accuracy: 0.9088 - val_loss: 0.5462 - val_accuracy: 0.8469
Epoch 85/200
132/132 [==============================] - ETA: 0s - loss: 0.3110 - accuracy: 0.9133
Epoch 00085: val_accuracy did not improve from 0.88818
132/132 [==============================] - 136s 1s/step - loss: 0.3110 - accuracy: 0.9133 - val_loss: 0.4943 - val_accuracy: 0.8255
Epoch 86/200
132/132 [==============================] - ETA: 0s - loss: 0.2948 - accuracy: 0.9178
Epoch 00086: val_accuracy did not improve from 0.88818

Epoch 112/200
132/132 [==============================] - ETA: 0s - loss: 0.2509 - accuracy: 0.9380
Epoch 00112: val_accuracy did not improve from 0.89316
132/132 [==============================] - 135s 1s/step - loss: 0.2509 - accuracy: 0.9380 - val_loss: 0.7835 - val_accuracy: 0.7813
Epoch 113/200
132/132 [==============================] - ETA: 0s - loss: 0.2653 - accuracy: 0.9309
Epoch 00113: val_accuracy improved from 0.89316 to 0.89530, saving model to ./logs/_Nov22_10-44-52/checkpoint/model_cp_113.h5
132/132 [==============================] - 137s 1s/step - loss: 0.2653 - accuracy: 0.9309 - val_loss: 0.3551 - val_accuracy: 0.8953
Epoch 114/200
132/132 [==============================] - ETA: 0s - loss: 0.2511 - accuracy: 0.9330
Epoch 00114: val_accuracy did not improve from 0.89530
132/132 [==============================] - 135s 1s/step - loss: 0.2511 - accuracy: 0.9330 - val_loss: 0.3643 - val_accuracy: 0.8917
Epoch 115/200
132/132 [==============================] - ETA: 0s - loss

Epoch 140/200
132/132 [==============================] - ETA: 0s - loss: 0.2235 - accuracy: 0.9487
Epoch 00140: val_accuracy improved from 0.90100 to 0.90670, saving model to ./logs/_Nov22_10-44-52/checkpoint/model_cp_140.h5
132/132 [==============================] - 141s 1s/step - loss: 0.2235 - accuracy: 0.9487 - val_loss: 0.3891 - val_accuracy: 0.9067
Epoch 141/200
132/132 [==============================] - ETA: 0s - loss: 0.2231 - accuracy: 0.9461
Epoch 00141: val_accuracy did not improve from 0.90670
132/132 [==============================] - 139s 1s/step - loss: 0.2231 - accuracy: 0.9461 - val_loss: 0.4367 - val_accuracy: 0.8839
Epoch 142/200
132/132 [==============================] - ETA: 0s - loss: 0.2306 - accuracy: 0.9387
Epoch 00142: val_accuracy did not improve from 0.90670
132/132 [==============================] - 139s 1s/step - loss: 0.2306 - accuracy: 0.9387 - val_loss: 0.4269 - val_accuracy: 0.8868
Epoch 143/200
132/132 [==============================] - ETA: 0s - loss

Epoch 169/200
132/132 [==============================] - ETA: 0s - loss: 0.1796 - accuracy: 0.9591
Epoch 00169: val_accuracy did not improve from 0.90670
132/132 [==============================] - 131s 996ms/step - loss: 0.1796 - accuracy: 0.9591 - val_loss: 0.3995 - val_accuracy: 0.8910
Epoch 170/200
132/132 [==============================] - ETA: 0s - loss: 0.1852 - accuracy: 0.9608
Epoch 00170: val_accuracy did not improve from 0.90670
132/132 [==============================] - 132s 1s/step - loss: 0.1852 - accuracy: 0.9608 - val_loss: 0.4073 - val_accuracy: 0.8675
Epoch 171/200
132/132 [==============================] - ETA: 0s - loss: 0.1858 - accuracy: 0.9572
Epoch 00171: val_accuracy did not improve from 0.90670
132/132 [==============================] - 134s 1s/step - loss: 0.1858 - accuracy: 0.9572 - val_loss: 0.4073 - val_accuracy: 0.8903
Epoch 172/200
132/132 [==============================] - ETA: 0s - loss: 0.1770 - accuracy: 0.9594
Epoch 00172: val_accuracy did not improv

In [ ]:
#--------------Saving the model---------------
#---------------------------------------------


from datetime import datetime

savedir = os.path.join('./savedModel'+ datetime.now().strftime('%b%d_%H-%M-%S'))

if not os.path.exists(savedir):
    os.makedirs(savedir) 

model.save(savedir)

In [6]:
clmode = "rgb"
source = '../input/artificial-neural-networks-and-deep-learning-2020/MaskDataset'

test_data_gen = ImageDataGenerator(rescale = 1./255)

test_datagen = test_data_gen.flow_from_directory(
    source,
    target_size = (256, 256),
    color_mode = clmode,
    classes =  ["test"],
    class_mode = "categorical",
    batch_size = 1,
    shuffle = False
)

test_datagen.reset()

Found 450 images belonging to 1 classes.


In [ ]:
predictions = model.predict_generator(test_datagen, len(test_datagen), verbose = 1)
result = {}

In [ ]:
import ntpath

images = test_datagen.filenames
i = 0

for p in predictions:
    prediction = np.argmax(p)
    image_name = ntpath.basename(images[i])
    result[image_name] = str(prediction)
    i = i+1


In [ ]:
def create_csv(results):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join('./', csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')

In [ ]:
create_csv(result)